### This is project is to predict the wellhead pressure and flow pattern within the well

In [4]:
import itertools
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from sklearn import preprocessing

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models

%matplotlib inline

In [37]:
df=pd.read_csv('simple_well1.csv')

In [38]:
df.head()

,Date,Gas Flow_MMSCFD,Oil_bpd,Water_bpd,FWHP_psig,FroudeNumberGas6,FroudeNumberLiquid6,HoldupFractionLiquid6,ReynoldsNumber6,SuperficialVelocityGas6,SuperficialVelocityLiquid6,SurfaceTensionLiquidInSitu6,ViscosityLiquidInSitu6,FlowPatternGasLiquid6,FINAPRESS,FINATEMP
0,1/1/2020,2.0,480.0,4500.0,5600.0,0.079350,0.829993,0.845727,409800.7890,0.510384,2.641022,0.013319,0.273293,4,2126.376735,212.370290
1,1/2/2020,2.0,480.0,3450.0,5500.0,0.079880,0.669710,0.817304,334632.3328,0.507737,2.124772,0.012765,0.275990,4,2177.457559,208.752810
2,1/3/2020,2.0,480.0,3000.0,5400.0,0.081080,0.598358,0.797930,303566.9168,0.517678,1.900507,0.012747,0.277711,4,2168.102001,206.531840
3,1/4/2020,2.0,479.0,2500.0,5300.0,0.082113,0.520704,0.774212,268085.3347,0.522150,1.652199,0.012434,0.280040,4,2183.207951,203.401306
4,1/5/2020,2.0,479.0,2000.0,5200.0,0.082792,0.444817,0.747499,231149.3853,0.519421,1.406630,0.011781,0.283581,4,2220.778037,199.208880


In [39]:
df.drop(columns='Date', inplace=True)

In [40]:
df.head()

,Gas Flow_MMSCFD,Oil_bpd,Water_bpd,FWHP_psig,FroudeNumberGas6,FroudeNumberLiquid6,HoldupFractionLiquid6,ReynoldsNumber6,SuperficialVelocityGas6,SuperficialVelocityLiquid6,SurfaceTensionLiquidInSitu6,ViscosityLiquidInSitu6,FlowPatternGasLiquid6,FINAPRESS,FINATEMP
0,2.0,480.0,4500.0,5600.0,0.079350,0.829993,0.845727,409800.7890,0.510384,2.641022,0.013319,0.273293,4,2126.376735,212.370290
1,2.0,480.0,3450.0,5500.0,0.079880,0.669710,0.817304,334632.3328,0.507737,2.124772,0.012765,0.275990,4,2177.457559,208.752810
2,2.0,480.0,3000.0,5400.0,0.081080,0.598358,0.797930,303566.9168,0.517678,1.900507,0.012747,0.277711,4,2168.102001,206.531840
3,2.0,479.0,2500.0,5300.0,0.082113,0.520704,0.774212,268085.3347,0.522150,1.652199,0.012434,0.280040,4,2183.207951,203.401306
4,2.0,479.0,2000.0,5200.0,0.082792,0.444817,0.747499,231149.3853,0.519421,1.406630,0.011781,0.283581,4,2220.778037,199.208880


In [41]:
df['FlowPatternGasLiquid6'].nunique()

2

In [42]:
df2=pd.get_dummies(df['FlowPatternGasLiquid6'],prefix='FlowPattern')

In [43]:
df2.head()

,FlowPattern_3,FlowPattern_4
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1


In [44]:
df=pd.concat([df,df2],axis=1)

In [45]:
df.drop(columns='FlowPatternGasLiquid6', inplace=True)

In [46]:
df.head()

,Gas Flow_MMSCFD,Oil_bpd,Water_bpd,FWHP_psig,FroudeNumberGas6,FroudeNumberLiquid6,HoldupFractionLiquid6,ReynoldsNumber6,SuperficialVelocityGas6,SuperficialVelocityLiquid6,SurfaceTensionLiquidInSitu6,ViscosityLiquidInSitu6,FINAPRESS,FINATEMP,FlowPattern_3,FlowPattern_4
0,2.0,480.0,4500.0,5600.0,0.079350,0.829993,0.845727,409800.7890,0.510384,2.641022,0.013319,0.273293,2126.376735,212.370290,0,1
1,2.0,480.0,3450.0,5500.0,0.079880,0.669710,0.817304,334632.3328,0.507737,2.124772,0.012765,0.275990,2177.457559,208.752810,0,1
2,2.0,480.0,3000.0,5400.0,0.081080,0.598358,0.797930,303566.9168,0.517678,1.900507,0.012747,0.277711,2168.102001,206.531840,0,1
3,2.0,479.0,2500.0,5300.0,0.082113,0.520704,0.774212,268085.3347,0.522150,1.652199,0.012434,0.280040,2183.207951,203.401306,0,1
4,2.0,479.0,2000.0,5200.0,0.082792,0.444817,0.747499,231149.3853,0.519421,1.406630,0.011781,0.283581,2220.778037,199.208880,0,1


In [70]:
y_pre=df['FINAPRESS']
X_pre=df.drop(columns=['FINAPRESS','FINATEMP'], axis=1)

In [71]:
y_hol=df['HoldupFractionLiquid6']
X_hol=df.drop(columns=['HoldupFractionLiquid6','FINATEMP'], axis=1)

In [72]:
y_pat=df[['FlowPattern_3','FlowPattern_4']]
X_pat=df.drop(columns=['FlowPattern_3','FlowPattern_4','FINATEMP'], axis=1)